#### Scaling main - Arnsberg

In [2]:
import pickle
import numpy as np
from scipy.linalg import null_space

In [3]:
# Load matrix from file
with open('matrix_arnsberg.pkl', 'rb') as f:
    Q = pickle.load(f)

print("Q matrix which we got from adjacencies:")
print(Q)

Q matrix which we got from adjacencies:
[[ 3 -1 -1  0  0 -1  0  0  0  0  0  0]
 [-1  4 -1 -1  0  0  0  0  0  0  0 -1]
 [-1 -1  4 -1  0  0  0 -1  0  0  0  0]
 [ 0 -1 -1  4  0  0  0 -1  0  0  0 -1]
 [ 0  0  0  0  2  0  0  0  0  0 -1 -1]
 [-1  0  0  0  0  1  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  4 -1 -1 -1 -1  0]
 [ 0  0 -1 -1  0  0 -1  6 -1  0 -1 -1]
 [ 0  0  0  0  0  0 -1 -1  3 -1  0  0]
 [ 0  0  0  0  0  0 -1  0 -1  2  0  0]
 [ 0  0  0  0 -1  0 -1 -1  0  0  4 -1]
 [ 0 -1  0 -1 -1  0  0 -1  0  0 -1  5]]


In [4]:
# get the machine epsilon
eps = np.finfo(float).eps   

# add small epsion value to the diagonal of the matrix
Q = Q + np.eye(Q.shape[0]) * np.sqrt(eps)

In [5]:
# get the determinant of the matrix
det = np.linalg.det(Q)
det

0.0023270847373585007

#### Compute the generalized inverse
##### The generalized inverse of a matrix, also known as the Moore-Penrose inverse (oseudo Inverse)

This code performs the following steps:

1. **Compute SVD of Q**: Singular value decomposition decomposes the matrix \( Q \) into three matrices \( U \), \( S \), and \( V^T \), such that \( Q = U \cdot \text{diag}(S) \cdot V^T \), where \( U \) and \( V \) are orthogonal matrices and \( S \) is a diagonal matrix containing the singular values.

2. **Set Threshold for Singular Values**: We set a threshold value to determine when a singular value is considered nonzero. This is necessary to avoid numerical instability issues.

3. **Calculate Inverse of Singular Values**: We calculate the inverse of the singular values, ensuring that we avoid dividing by very small values close to zero.

4. **Construct the Generalized Inverse**: Using the inverse of singular values obtained in the previous step, we construct the generalized inverse of \( Q \) using the formula \( Q^{-} = V \cdot \text{diag}(1/S_{\text{inv}}) \cdot U^T \).

The resulting matrix \( Q^{-} \) is the generalized inverse of \( Q \).

In [6]:
# Compute SVD of Q
U, S, Vt = np.linalg.svd(Q)

In [7]:
# get the machine epsilon
eps = np.finfo(float).eps
tol = np.sqrt(eps)
tol

1.4901161193847656e-08

In [8]:
# Calculate inverse of singular values
S_inv = np.where(S > tol, 1/S, 0)

In [9]:
# Construct the generalized inverse
Q_inv = Vt.T @ np.diag(S_inv) @ U.T

In [10]:
print("Generalized Inverse of Q:")
print(Q_inv)

Generalized Inverse of Q:
[[ 0.52957871  0.11550541  0.11031869 -0.01208794 -0.16325207  0.44624537
  -0.20543741 -0.10838808 -0.21661767 -0.25269421 -0.16071634 -0.08245447]
 [ 0.11550541  0.30516655  0.09251095  0.07383875 -0.06833506  0.03217208
  -0.13126729 -0.04113359 -0.1452138  -0.17990721 -0.07548121  0.02214442]
 [ 0.11031869  0.09251095  0.2947931   0.06865203 -0.09150242  0.02698536
  -0.11294087 -0.00897591 -0.12135488 -0.15881454 -0.07928481 -0.0203867 ]
 [-0.01208794  0.07383875  0.06865203  0.27957872 -0.03825207 -0.09542128
  -0.08043741  0.01661192 -0.09161768 -0.12769421 -0.03571634  0.04254553]
 [-0.16325207 -0.06833506 -0.09150242 -0.03825207  0.57914073 -0.2465854
  -0.03150934 -0.01283714 -0.07404046 -0.09444156  0.13861514  0.10299965]
 [ 0.44624537  0.03217208  0.02698536 -0.09542128 -0.2465854   1.36291202
  -0.28877074 -0.19172141 -0.299951   -0.33602753 -0.24404967 -0.1657878 ]
 [-0.20543741 -0.13126729 -0.11294087 -0.08043741 -0.03150934 -0.28877074
   0.36

### Compute generalized variance - using Q Inv

In [11]:
generalized_variance = np.exp(np.mean(np.log(np.diag(Q_inv))))  # equation in the paper use daba as 1
generalized_variance #fac in R code

0.4001011249419349

In [12]:
marginal_variances = np.diag(Q_inv/generalized_variance)
marginal_variances

array([1.32361215, 0.76272354, 0.73679649, 0.69877014, 1.44748587,
       3.40641886, 0.90474619, 0.45073465, 1.20607618, 1.80239843,
       0.75840237, 0.58036993])

In [13]:
geometric_mean = np.exp(np.mean(np.log(marginal_variances)))
geometric_mean

0.9999999999999999

In [14]:
QQ = generalized_variance*Q # this is completely different!
print(np.diag(QQ))

[1.20030338 1.60040451 1.60040451 1.60040451 0.80020226 0.40010113
 1.60040451 2.40060676 1.20030338 0.80020226 1.60040451 2.00050563]
